In [1]:
import sys
sys.path.insert(0, '../')
import numpy as np
%reload_ext autoreload
%autoreload 1
%aimport allel
%aimport allel.model
%aimport allel.model.ndarray
%aimport allel.model.chunked
allel.__version__

'0.20.2'

In [2]:
# setup an array of genotype calls
shape = n_variants, n_samples, ploidy = 50000, 1000, 2
data = np.zeros(shape, dtype='i1')
# simulate some mutations
n_alleles = n_variants * n_samples * ploidy
idx = np.random.randint(0, (n_alleles//2)-1, size=n_alleles//20)
data[:, :, 1].reshape((-1))[idx] = 1
data[:, :, 0].reshape((-1))[idx[:n_alleles//200]] = 1
g = allel.GenotypeArray(data, copy=False)

In [3]:
g

,0,1,2,3,4,...,995,996,997,998,999
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
1,0/0,0/0,0/0,0/0,0/0,...,0/0,1/1,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
4,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,1/1,0/0


In [4]:
g.take([0, 1, 2], axis=1)

,0,1,2
0,0/0,0/0,0/0
1,0/0,0/0,0/0
2,0/0,0/0,0/0
3,0/0,0/0,0/0
4,0/0,0/0,0/0


In [5]:
h = g.to_haplotypes()
h

,0,1,2,3,4,...,1995,1996,1997,1998,1999
0,0,0,0,0,0,...,0,0,0,0,0
1,0,0,0,0,0,...,0,0,0,0,0
2,0,0,0,0,0,...,0,0,0,0,0
3,0,0,0,0,0,...,0,0,0,0,0
4,0,0,0,0,0,...,0,1,1,0,0


In [6]:
h.take([0, 1, 2], axis=1)

,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [7]:
gc = allel.GenotypeChunkedArray(g).copy()
gc

,0,1,2,3,4,...,995,996,997,998,999
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
1,0/0,0/0,0/0,0/0,0/0,...,0/0,1/1,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
4,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,1/1,0/0


In [8]:
gc.take([0, 1, 2], axis=1)

,0,1,2
0,0/0,0/0,0/0
1,0/0,0/0,0/0
2,0/0,0/0,0/0
3,0/0,0/0,0/0
4,0/0,0/0,0/0


In [9]:
hc = gc.to_haplotypes()
hc

,0,1,2,3,4,...,1995,1996,1997,1998,1999
0,0,0,0,0,0,...,0,0,0,0,0
1,0,0,0,0,0,...,0,0,0,0,0
2,0,0,0,0,0,...,0,0,0,0,0
3,0,0,0,0,0,...,0,0,0,0,0
4,0,0,0,0,0,...,0,1,1,0,0


In [10]:
hc.take([0, 1, 2], axis=1)

,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [11]:
ac = g.count_alleles()
ac

,0,1
0,1911,89
1,1895,105
2,1896,104
3,1892,108
4,1892,108


In [12]:
acc = gc.count_alleles()
acc

,0,1
0,1911,89
1,1895,105
2,1896,104
3,1892,108
4,1892,108


In [13]:
gc.is_called()

carray((50000, 1000), bool)
  nbytes: 47.68 MB; cbytes: 743.19 KB; ratio: 65.70
  cparams := cparams(clevel=5, shuffle=True, cname='blosclz')
[[ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 ..., 
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]]

In [14]:
gc.to_n_alt()

ChunkedArray((50000, 1000), int8, nbytes=47.7M, cbytes=13.0M, cratio=3.7, cname=blosclz, clevel=5, shuffle=True, chunks=(524, 1000), data=bcolz.carray_ext.carray)

In [15]:
pos = np.unique(np.random.randint(0, 1000000, size=50000))
pos

array([     9,     28,     30, ..., 999946, 999958, 999960])

In [16]:
ref = np.random.choice([b'A', b'C', b'T', b'G'], size=pos.shape[0])
ref

array([b'A', b'T', b'A', ..., b'G', b'G', b'A'], 
      dtype='|S1')

In [17]:
alt = np.random.choice([b'A', b'C', b'T', b'G'], size=pos.shape[0])
alt

array([b'G', b'A', b'A', ..., b'A', b'T', b'A'], 
      dtype='|S1')

In [18]:
ra = np.rec.fromarrays([pos, ref, alt], names=['POS', 'REF', 'ALT'])
vtbl = allel.VariantTable(ra)
vtbl

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'


In [19]:
vctbl = allel.VariantChunkedTable(vtbl).copy()
vctbl

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'


In [20]:
vtbl[:5].display(10)

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'


In [21]:
vtbl[:5].display(None)

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'


In [22]:
vtbl[:5].displayall()

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'


In [23]:
vctbl.take(list(range(5))).display(10)

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'


In [24]:
vctbl.take(list(range(5))).display(None)

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'


In [25]:
vctbl.take(list(range(5))).displayall()

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'
